<a href="https://colab.research.google.com/github/ArpanSaha07/signlang-to-text/blob/main/signlang_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the start of our sign language detection project. Authors: Emilia and Arpan

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam

from sklearn.metrics import classification_report,confusion_matrix

import tensorflow as tf

import cv2
import os

import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/signlang_detect/' 

# change the runtime type to GPU

Mounted at /content/gdrive


Step 2: Loading the data

In [ ]:
labels_alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','nothing','del','space']
labels_numbers = ['0','1','2','3','4','5','6','7','8','9']
img_size = 500

def get_data(data_dir):
  data = []
  for label in labels_numbers: 
    path = os.path.join(data_dir, label)
    class_num = labels_numbers.index(label)
    for img in os.listdir(path):
      try:
        img_arr = cv2.imread(os.path.join(path, img))[...,::-1] #convert BGR to RGB format
        resized_arr = cv2.resize(img_arr, (img_size, img_size)) # Reshaping images to preferred size
        data.append([resized_arr, class_num])
      except Exception as e:
        print(e)
  return np.array(data)
  
train_numbers = get_data('../rootpath/train')  #need to change address
val_numbers = get_data('../rootpath/')



Step 3: Visualize the data

In [ ]:
#This block of code helps us visualize our dataset and see how many images of every label do we have
occurance = []
for img_number in train_numbers:
  occurance.append(img_number[1]) #this append the label of the image
sns.set_style('darkgrid')
sns.countplot(occurance)

#This is just fun because we can visualize images from the dataset if we need
plt.figure(figsize = (5,5))
plt.imshow(train_numbers[1][0]) #train_numbers[1][0] gives the jpeg file
plt.title(labels_numbers[train_numbers[0][1]]) #train_numbers[0][1] gives the label of the image

Step 4: Data augmentation

In [ ]:
x_nb_train = []
y_nb_train = []
x_nb_val = []
y_nb_val = []

for feature, label in train_numbers:
  x_nb_train.append(feature)
  y_nb_train.append(label)

for feature, label in val_numbers:
  x_nb_val.append(feature)
  y_nb_val.append(label)

# Normalize the data
x_nb_train = np.array(x_nb_train) / 255
x_nb_val = np.array(x_nb_val) / 255

x_nb_train.reshape(-1, img_size, img_size, 1)
y_train = np.array(y_nb_train)

x_nb_val.reshape(-1, img_size, img_size, 1)
y_nb_val = np.array(y_nb_val)


datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range = 30,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.2, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip = True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


datagen.fit(x_nb_train)



Step 5: Defining our model

In [ ]:
model = Sequential()
model.add(Conv2D(32,3,padding="same", activation="relu", input_shape=(img_size,img_size,3)))
model.add(MaxPool2D())

model.add(Conv2D(32, 3, padding="same", activation="relu"))
model.add(MaxPool2D())
model.add(Dropout(0.4))

model.add(Flatten())
model.add(Dense(128,activation="relu"))
model.add(Dense(2, activation="softmax")) 

opt = Adam(lr=0.000001)
model.compile(optimizer = opt , loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True) , metrics = ['accuracy'])

history = model.fit(x_nb_train,y_nb_train,epochs = 500 , validation_data = (x_nb_val, y_nb_val))


Step 6: Visualizing our results

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(500)

plt.figure(figsize=(15, 15))
plt.subplot(2, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()